In [1]:
import pandas as pd

In [4]:
coffee_shop_df = pd.read_csv("coffee_shop_sales.csv", parse_dates=['date'])
coffee_shop_df.head()

,order_id,date,store,product,quantity,sales
0,1,2023-01-01,Lower Manhattan,Gourmet brewed coffee,2,6.0
1,2,2023-01-01,Lower Manhattan,Brewed Chai tea,2,6.2
2,3,2023-01-01,Lower Manhattan,Hot chocolate,2,9.0
3,4,2023-01-01,Lower Manhattan,Drip coffee,1,2.0
4,5,2023-01-01,Lower Manhattan,Brewed Chai tea,2,6.2


In [5]:
coffee_shop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   order_id  149116 non-null  int64         
 1   date      149116 non-null  datetime64[ns]
 2   store     149116 non-null  object        
 3   product   149116 non-null  object        
 4   quantity  149116 non-null  int64         
 5   sales     149116 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 6.8+ MB


In [18]:
summary = coffee_shop_df.groupby(['store', coffee_shop_df['date'].dt.month]).agg(
    sales_total = ('sales', 'sum')
).reset_index()
summary

,store,date,sales_total
0,Astoria,1,27313.66
1,Astoria,2,25105.34
2,Astoria,3,32835.43
3,Astoria,4,39477.61
4,Astoria,5,52428.76
5,Astoria,6,55083.11
6,Hell's Kitchen,1,27820.65
7,Hell's Kitchen,2,25719.80
8,Hell's Kitchen,3,33110.57
9,Hell's Kitchen,4,40304.14


In [24]:
prev_month_summary = coffee_shop_df.groupby(['store', coffee_shop_df['date'].dt.month]).agg(
    sales_total = ('sales', 'sum')
).shift(1).reset_index()
prev_month_summary

,store,date,sales_total
0,Astoria,1,NaN
1,Astoria,2,27313.66
2,Astoria,3,25105.34
3,Astoria,4,32835.43
4,Astoria,5,39477.61
5,Astoria,6,52428.76
6,Hell's Kitchen,1,55083.11
7,Hell's Kitchen,2,27820.65
8,Hell's Kitchen,3,25719.80
9,Hell's Kitchen,4,33110.57


In [51]:
result = pd.merge(
    prev_month_summary,
    summary,
    on=['store', 'date'],
    suffixes=["_prev", "_curr"]
)
result

,store,date,sales_total_prev,sales_total_curr
0,Astoria,1,NaN,27313.66
1,Astoria,2,27313.66,25105.34
2,Astoria,3,25105.34,32835.43
3,Astoria,4,32835.43,39477.61
4,Astoria,5,39477.61,52428.76
5,Astoria,6,52428.76,55083.11
6,Hell's Kitchen,1,55083.11,27820.65
7,Hell's Kitchen,2,27820.65,25719.80
8,Hell's Kitchen,3,25719.80,33110.57
9,Hell's Kitchen,4,33110.57,40304.14


In [52]:
result['profit'] = result['sales_total_curr'] - result['sales_total_prev']
result.rename(columns=
              {"date" : "month", 
               "sales_total_curr" : "sales", 
               "profit" : "Sales vs last month"}, inplace=True)
result = result.drop("sales_total_prev", axis=1)
result

,store,month,sales,Sales vs last month
0,Astoria,1,27313.66,NaN
1,Astoria,2,25105.34,-2208.32
2,Astoria,3,32835.43,7730.09
3,Astoria,4,39477.61,6642.18
4,Astoria,5,52428.76,12951.15
5,Astoria,6,55083.11,2654.35
6,Hell's Kitchen,1,27820.65,-27262.46
7,Hell's Kitchen,2,25719.80,-2100.85
8,Hell's Kitchen,3,33110.57,7390.77
9,Hell's Kitchen,4,40304.14,7193.57
